In [1]:
import torch
import torch.nn as nn
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

from utils import *
import resnet_cifar10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Load Dataset

In [2]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

url = "https://raw.githubusercontent.com/Mohamed2519/Text-Classification-For-SST2-dataset/main/data/test.tsv"
df = pd.read_csv(url, delimiter='\t', names=['labels','sentence'])

encoded_data = tokenizer.batch_encode_plus(
    df['sentence'].tolist(),
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

# Create TensorDataset from encoded_data
dataset = TensorDataset(
    encoded_data['input_ids'].to(device),
    encoded_data['attention_mask'].to(device),
    torch.tensor(df['labels'].tolist()).to(device)
)

# Create DataLoader from TensorDataset
batch_size = 32
testloaders = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Load Model

In [3]:
models = []
ids = [40, "45"]
criterion = nn.CrossEntropyLoss()
model_name ="dtl"
models_path = "distilbert_models/"

# load all models
m = len(ids)
for i in range(m):   
    model_name_path = model_name +"_"+str(ids[i])
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
    
    print("LOAD",models_path + model_name_path)
    model.load_state_dict(torch.load(models_path + model_name_path + ".pt", map_location=device))
    model.eval()
    model.to(device)
    models.append(model)

ens_evaluate_acc_distilbert(models=models, testloaders=testloaders)

LOAD distilbert_models/dtl_40
LOAD distilbert_models/dtl_45


0.9242174629324547

In [4]:
ens_evaluate_acc_distilbert(models=models, testloaders=testloaders) # warm up
ens_evaluate_acc_distilbert(models=models, testloaders=testloaders, single=True) # warm up
ens_evaluate_acc_distilbert(models=models, testloaders=testloaders, single=True, full_profiler=True)

c:\Program Files\Python39\lib\site-packages\torch\profiler\profiler.py:403: UserWarning: use_cuda is deprecated, use activities argument instead
  warn("use_cuda is deprecated, use activities argument instead")


-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  Total MFLOPs  
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
            model_inference_0        30.03%      19.471ms        97.82%      63.416ms      63.416ms      10.844ms        16.52%      64.232ms      64.232ms             1            --  
                 aten::linear         8.46%       5.484ms        27.76%      17.996ms     473.579us       2.455ms         3.74%      27.953ms     735.605us            38            --  
                  aten::addmm        11.37%       7.374ms        13.7

(tensor([0], device='cuda:0'),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'),
 [1682461542182.433, 1682461542485.1057, 598.0358123779297])